In [1]:
from pandas import read_csv, DataFrame, Index, concat

In [2]:
exchanges = ['Huobi', 'Deribit', 'Bybit', 'Bitmex', 'OKX', 'Binance', 'Bitget', 'Kraken']

In [3]:
huobi_data = read_csv('data/btc inverse perps/Huobi_1d.csv', parse_dates=['timestamp'])
huobi_data = huobi_data.drop_duplicates(subset='timestamp', keep="last")
huobi_data = huobi_data.set_index('timestamp')
huobi_data.name = 'Huobi'

binance_data = read_csv('data/btc inverse perps/Binance_1d.csv', parse_dates=['timestamp'])
binance_data = binance_data.drop_duplicates(subset='timestamp', keep="last")
binance_data = binance_data.set_index('timestamp')
binance_data.name = 'Binance'

deribit_data = read_csv('data/btc inverse perps/Deribit_1d.csv', parse_dates=['timestamp'])
deribit_data = deribit_data.drop_duplicates(subset='timestamp', keep="last")
deribit_data = deribit_data.set_index('timestamp')
deribit_data = deribit_data.iloc[2:,:]
deribit_data.name = 'Deribit'

bybit_data = read_csv('data/btc inverse perps/Bybit_1d.csv', parse_dates=['timestamp'])
bybit_data = bybit_data.drop_duplicates(subset='timestamp', keep="last")
bybit_data = bybit_data.set_index('timestamp')
bybit_data = bybit_data.iloc[2:,:]
bybit_data.name = 'Bybit'

bitget_data = read_csv('data/btc inverse perps/Bitget_1d.csv', parse_dates=['timestamp'])
bitget_data = bitget_data.drop_duplicates(subset='timestamp', keep="last")
bitget_data = bitget_data.set_index('timestamp')
bitget_data.name = 'Bitget'

bitmex_data = read_csv('data/btc inverse perps/Bitmex_1d.csv', parse_dates=['timestamp'])
bitmex_data = bitmex_data.drop_duplicates(subset='timestamp', keep="last")
bitmex_data = bitmex_data.set_index('timestamp')
bitmex_data = bitmex_data.iloc[2:,:]
bitmex_data.name = 'Bitmex'

okx_data = read_csv('data/btc inverse perps/OKX_1d.csv', parse_dates=['timestamp'])
okx_data = okx_data.drop_duplicates(subset='timestamp', keep="last")
okx_data = okx_data.set_index('timestamp')
okx_data = okx_data.iloc[2:,:]
okx_data.name = 'OKX'

kraken_data = read_csv('data/btc inverse perps/Kraken_1d.csv', parse_dates=['timestamp'])
kraken_data = kraken_data.drop_duplicates(subset='timestamp', keep="last")
kraken_data = kraken_data.set_index('timestamp')
kraken_data.name = 'Kraken'

In [4]:
dfs_2020Q2_2023Q1 = [huobi_data, deribit_data, bybit_data, bitmex_data, okx_data]
dfs_2023Q2 = [huobi_data, deribit_data, bybit_data, bitmex_data, okx_data, bitget_data, kraken_data]
dfs_2023Q3_2023Q4 = [huobi_data, deribit_data, bybit_data, bitmex_data, okx_data, binance_data, bitget_data, kraken_data]

In [5]:
def agg_OI_quarterly(df):
    ret = df['OI Close'].resample('Q').sum()
    ret = ret.to_frame(name='Agg OI').reset_index()
    ret['Quarter'] = ret['timestamp'].dt.to_period('Q')
    ret.name = df.name
    return ret

In [6]:
dfs_2020Q2_2023Q1_quarterly_OI = [agg_OI_quarterly(df) for df in dfs_2020Q2_2023Q1]
dfs_2023Q2_quarterly_OI = [agg_OI_quarterly(df) for df in dfs_2023Q2]
dfs_2023Q3_2023Q4 = [agg_OI_quarterly(df) for df in dfs_2023Q3_2023Q4]

In [7]:
def get_ratio(df, dfs, quarter):
    sum_of_quarter = 0
    this_value = df.loc[df['Quarter'] == quarter]['Agg OI'].values[0]
    for _df in dfs:
        sum_of_quarter += _df.loc[_df['Quarter'] == quarter]['Agg OI'].values[0]
    return df.name, quarter, this_value / sum_of_quarter

In [8]:
quarter_list = [
    '2020Q2', '2020Q3', '2020Q4', '2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1',
    '2022Q2', '2022Q3', '2022Q4', '2023Q1', '2023Q2', '2023Q3', '2023Q4'
]

In [9]:
ratios_by_quarter = []

for quarter in quarter_list:
    ratios = []

    if quarter == '2023Q2':
        for df in dfs_2023Q2_quarterly_OI:
            _, _, ratio = get_ratio(df, dfs_2023Q2_quarterly_OI, quarter)
            ratios.append(ratio)
    elif quarter in ['2023Q3', '2023Q4']:
        for df in dfs_2023Q3_2023Q4:
            _, _, ratio = get_ratio(df, dfs_2023Q3_2023Q4, quarter)
            ratios.append(ratio)
    else:
        for df in dfs_2020Q2_2023Q1_quarterly_OI:
            _, _, ratio = get_ratio(df, dfs_2020Q2_2023Q1_quarterly_OI, quarter)
            ratios.append(ratio)

    ratios_by_quarter.append(ratios)

In [10]:
for i in range(len(ratios_by_quarter)):
    if i <= 11:
        ratios_by_quarter[i].insert(5, 0)
        ratios_by_quarter[i].insert(6, 0)
        ratios_by_quarter[i].insert(7, 0)
    if i == 12:
        ratios_by_quarter[i].insert(5, 0)

In [11]:
oi_ratios_by_quarter_and_exchange = DataFrame(
    ratios_by_quarter,
    columns=exchanges
)

oi_ratios_by_quarter_and_exchange = oi_ratios_by_quarter_and_exchange.set_index(Index(quarter_list))
oi_ratios_by_quarter_and_exchange['Checksum'] = oi_ratios_by_quarter_and_exchange.sum(axis=1)

In [12]:
oi_ratios_by_quarter_and_exchange

,Huobi,Deribit,Bybit,Bitmex,OKX,Binance,Bitget,Kraken,Checksum
2020Q2,0.092678,0.075095,0.172987,0.554620,0.104620,0.000000,0.000000,0.000000,1.0
2020Q3,0.153404,0.072746,0.213648,0.467397,0.092805,0.000000,0.000000,0.000000,1.0
2020Q4,0.144935,0.111903,0.361464,0.274520,0.107177,0.000000,0.000000,0.000000,1.0
2021Q1,0.128307,0.139291,0.455804,0.189173,0.087424,0.000000,0.000000,0.000000,1.0
2021Q2,0.125797,0.153031,0.455724,0.175106,0.090341,0.000000,0.000000,0.000000,1.0
2021Q3,0.091755,0.180789,0.413364,0.228944,0.085148,0.000000,0.000000,0.000000,1.0
2021Q4,0.047300,0.193181,0.484304,0.157884,0.117330,0.000000,0.000000,0.000000,1.0
2022Q1,0.048794,0.237229,0.373900,0.187355,0.152722,0.000000,0.000000,0.000000,1.0
2022Q2,0.036651,0.260827,0.323576,0.164361,0.214584,0.000000,0.000000,0.000000,1.0
2022Q3,0.032399,0.263301,0.268561,0.175353,0.260385,0.000000,0.000000,0.000000,1.0


In [13]:
def get_weighted_fr(exchange_data):
    exchange_name = exchange_data.name
    exchange_quarter_weights = dict(oi_ratios_by_quarter_and_exchange[exchange_name])

    ret = exchange_data.copy()
    ret = ret.reset_index()
    ret['Quarter'] = ret['timestamp'].dt.to_period('Q')
    ret[f'{exchange_name} Weighted FR'] = ret['FR Close'] * ret['Quarter'].replace(exchange_quarter_weights)
    
    ret = ret.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'OI Open', 'OI High',
                    'OI Low', 'FR Open', 'FR High', 'FR Low', 'FR Close', 'Quarter'], axis=1)
    ret.name = exchange_name
    return ret

In [14]:
huobi_wfr = get_weighted_fr(huobi_data)
huobi_wfr

,timestamp,OI Close,Huobi Weighted FR
0,2020-04-13,21751200.0,-0.002083
1,2020-04-14,22565500.0,-0.000382
2,2020-04-15,25736600.0,0.000927
3,2020-04-16,24025400.0,0.000927
4,2020-04-17,24886900.0,0.000927
...,...,...,...
1353,2023-12-27,48203100.0,0.000665
1354,2023-12-28,49419500.0,0.001392
1355,2023-12-29,50878300.0,0.001356
1356,2023-12-30,47641100.0,0.000910


In [15]:
deribit_wfr = get_weighted_fr(deribit_data)
deribit_wfr

,timestamp,OI Close,Deribit Weighted FR
0,2020-04-01,61702400.0,0.000000
1,2020-04-02,51202070.0,0.000000
2,2020-04-03,52880300.0,0.000000
3,2020-04-04,52814030.0,0.000000
4,2020-04-05,53506950.0,0.000000
...,...,...,...
1365,2023-12-27,652122360.0,0.011601
1366,2023-12-28,639148530.0,0.011963
1367,2023-12-29,593720630.0,0.008750
1368,2023-12-30,597387600.0,0.011214


In [16]:
bybit_wfr = get_weighted_fr(bybit_data)
bybit_wfr

,timestamp,OI Close,Bybit Weighted FR
0,2020-04-01,104986920.0,-0.006902
1,2020-04-02,92831162.0,0.001730
2,2020-04-03,92653280.0,0.001730
3,2020-04-04,99234996.0,0.001540
4,2020-04-05,99488446.0,0.001211
...,...,...,...
1365,2023-12-27,625769527.0,0.002024
1366,2023-12-28,620058811.0,0.005263
1367,2023-12-29,609075657.0,0.002024
1368,2023-12-30,636570636.0,0.002024


In [17]:
bitmex_wfr = get_weighted_fr(bitmex_data)
bitmex_wfr

,timestamp,OI Close,Bitmex Weighted FR
0,2020-04-01,453000958.0,-0.029506
1,2020-04-02,443449383.0,0.005546
2,2020-04-03,460430419.0,-0.024015
3,2020-04-04,455545925.0,-0.008375
4,2020-04-05,443729113.0,0.000388
...,...,...,...
1365,2023-12-27,284021968.0,0.004106
1366,2023-12-28,263685308.0,0.002438
1367,2023-12-29,274839373.0,0.000964
1368,2023-12-30,267238878.0,0.001012


In [18]:
okx_wfr = get_weighted_fr(okx_data)
okx_wfr

,timestamp,OI Close,OKX Weighted FR
0,2020-04-01,76914300.0,-0.002665
1,2020-04-02,79019400.0,-0.000756
2,2020-04-03,77772600.0,0.000755
3,2020-04-04,78922300.0,-0.001227
4,2020-04-05,79012200.0,0.000098
...,...,...,...
1364,2023-12-27,476537400.0,0.008144
1365,2023-12-28,484760300.0,0.004067
1366,2023-12-29,467534300.0,0.002833
1367,2023-12-30,481244800.0,0.001876


In [19]:
binance_wfr = get_weighted_fr(binance_data)
binance_wfr

,timestamp,OI Close,Binance Weighted FR
0,2023-08-01,600321100.0,0.001764
1,2023-08-02,584579100.0,0.001764
2,2023-08-03,583763500.0,0.001764
3,2023-08-04,598047100.0,0.001764
4,2023-08-05,598540600.0,0.001764
...,...,...,...
148,2023-12-27,868000300.0,0.002372
149,2023-12-28,878919000.0,0.004458
150,2023-12-29,872775000.0,0.003013
151,2023-12-30,886603700.0,0.006443


In [20]:
bitget_wfr = get_weighted_fr(bitget_data)
bitget_wfr

,timestamp,OI Close,Bitget Weighted FR
0,2023-04-22,2373129.0,0.000000
1,2023-04-23,2514954.0,0.000000
2,2023-04-24,2411606.0,0.000000
3,2023-04-25,2663041.0,0.000000
4,2023-04-26,2684988.0,0.000000
...,...,...,...
249,2023-12-27,175269298.0,0.000590
250,2023-12-28,182277017.0,0.001092
251,2023-12-29,179303504.0,0.000739
252,2023-12-30,167372258.0,0.001516


In [21]:
kraken_wfr = get_weighted_fr(kraken_data)
kraken_wfr

,timestamp,OI Close,Kraken Weighted FR
0,2023-05-20,27926777.0,-0.000012
1,2023-05-21,29301603.0,0.000008
2,2023-05-22,27439846.0,-0.000009
3,2023-05-23,27469237.0,0.000028
4,2023-05-24,30134541.0,0.000022
...,...,...,...
221,2023-12-27,17590938.0,0.000019
222,2023-12-28,17424108.0,0.000030
223,2023-12-29,17823801.0,0.000014
224,2023-12-30,17585983.0,0.000024


In [26]:
def get_agg_weighted_fr_oi(wfr_dfs):
    agg_wfr = []
    agg_oi_ = []

    for ts in bitmex_wfr['timestamp']:
        agg_fr = 0
        agg_oi = 0
        for exchange in wfr_dfs:
            row = exchange.loc[exchange['timestamp'] == ts]
            if row.size > 0:
                agg_oi += row['OI Close'].values[0]
                agg_fr += row[f'{exchange.name} Weighted FR'].values[0]
            else:
                continue
        agg_wfr.append(agg_fr)
        agg_oi_.append(agg_oi)

    ret = bitmex_wfr.copy()
    ret = ret.drop(['Bitmex Weighted FR', 'OI Close'], axis=1)
    ret['Agg Weighted FR'] = agg_wfr
    ret['Agg OI'] = agg_oi_

    ret = ret.set_index('timestamp')
    
    return ret

In [27]:
agg = get_agg_weighted_fr_oi([huobi_wfr, deribit_wfr, bybit_wfr, bitmex_wfr, okx_wfr, binance_wfr, bitget_wfr, kraken_wfr])
agg

,Agg Weighted FR,Agg OI
timestamp,,
2020-04-01,-0.039073,6.966046e+08
2020-04-02,0.006521,6.665020e+08
2020-04-03,-0.021530,6.837366e+08
2020-04-04,-0.008062,6.865173e+08
2020-04-05,0.001697,6.757367e+08
...,...,...
2023-12-27,0.029520,3.147515e+09
2023-12-28,0.030703,3.135693e+09
2023-12-29,0.019693,3.065951e+09
